# Music Lyrics Through the Years

#### Import (possibly) necessary libraries

In [1]:
import pandas as pd; pd.options.display.max_rows = 200
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('ticks')

import re
from unicodedata import normalize

import requests as rq
from bs4 import BeautifulSoup as bs
from selenium import webdriver

import pickle
import gzip

from functions.webscraping import *

import json

%load_ext autoreload
%autoreload 2

## Scrape song title and artist

In [2]:
%%time

# list of unique songs in top 10 between 1960 and 2019
all_top10s = [
    yearly_top10s(f'https://top40weekly.com/{year}-all-charts/') for year in range(1960,2020)
]

# confirm 60 years
len(all_top10s)

CPU times: user 14.7 s, sys: 195 ms, total: 14.9 s
Wall time: 1min 51s


60

In [3]:
# number of songs, before removing some duplicates across years
sum([len(year) for year in all_top10s])

5293

In [4]:
# uncomment to save
with gzip.open('data/all_top10s.pkl', 'wb') as goodbye:
    pickle.dump(all_top10s, goodbye, protocol=pickle.HIGHEST_PROTOCOL)
    
# uncomment to load
with gzip.open('data/all_top10s.pkl', 'rb') as hello:
    all_top10s = pickle.load(hello)

- Since some songs may be within the top 10 in two different years, I need to remove those from the list.

In [5]:
top10s = []

for i,year in enumerate(all_top10s):
    
    for song in year:
        
        song_dict = {}
        song_dict['year'] = i+1960
        song_dict['title'] = song[0]
        song_dict['artist'] = song[1]
        top10s.append(song_dict)

In [6]:
temp_df = pd.DataFrame(top10s)
temp_df = temp_df.drop_duplicates(subset=['title', 'artist'])

unique_top10s = temp_df.to_dict('records')

In [7]:
len(unique_top10s)

4735

## Scrape lyrics

In [8]:
# parse personal api keys
with open('data/api_keys.json', 'r') as json_file:
    creds = json.load(json_file)

# set the credentials based on the credentials file
client_id = creds['client_id']
client_secret = creds['client_secret']
access_token = creds['access_token']

In [9]:
%%time

top10s = []
print('Failed grabs:')

for song in unique_top10s:
    
    search_term = f"{song['title']} {song['artist']}"
        
    try:
        song['lyrics'] = lyrics_grabber(access_token, search_term)

    except:
        try:
            search_term = f"{re.match('[A-Z,. ]+', song['title']).group()} {' '.join(song['artist'].split()[:2])}"
            song['lyrics'] = lyrics_grabber(access_token, search_term)

        except:
            print(search_term)
            song['lyrics'] = np.nan
            
    top10s.append(song)
        
len(top10s)

Failed grabs:
(Down At) PAPA JOE’S The Dixiebelles
AN OPEN LETTER TO MY TEENAGE SON Victor Lundberg
/ 14 DOWN ON THE CORNER / FORTUNATE SON Creedence Clearwater Revival
Timeout raised and caught:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)
MIDNIGHT COWBOY Ferrante and
JUNGLE FEVER The Chakachas
YOU AIN Bachman-Turner Overdrive
JUJU ON DAT BEAT  Zayion McCall
CPU times: user 5min 53s, sys: 15.4 s, total: 6min 9s
Wall time: 1h 57min 57s


4735

In [10]:
df = pd.DataFrame(top10s)
df.shape

(4735, 4)

In [11]:
df.head()

,year,title,artist,lyrics
0,1960,THE VILLAGE OF ST. BERNADETTE,Andy Williams,"I've traveled far, the land and the sea\nBeaut..."
1,1960,PAPER ROSES,Anita Bryant,I realize the way your eyes deceived me\nWith ...
2,1960,IN MY LITTLE CORNER OF THE WORLD,Anita Bryant,"Oh, come along with me\nTo my little corner of..."
3,1960,O DIO MIO,Annette,"[Strofa 1]\nE un giorno lui mi disse: ""Nei tuo..."
4,1960,WONDERLAND BY NIGHT,Bert Kaempfert,Stars hang suspended\nAbove a floating yellow ...


In [12]:
df.tail()

,year,title,artist,lyrics
4730,2019,YOU NEED TO CALM DOWN,Taylor Swift,[Verse 1]\nYou are somebody that I don't know\...
4731,2019,ME!,Taylor Swift featuring Brendon Urie,"JANUARY""Giant"" by Calvin Harris featuring Rag ..."
4732,2019,HEARTLESS,The Weeknd,"[Intro]\nYoung Metro, young Metro, young Metro..."
4733,2019,DANCE MONKEY,Tones and I,"[Verse 1]\nThey say, ""Oh my God, I see the way..."
4734,2019,HIGHEST IN THE ROOM,Travis Scott,[Chorus]\nI got room in my fumes (Yeah)\nShe f...


### Remove brackets and words contained within
- These tend to be denoting verse from chorus (or something similar) or other non-lyric words.

In [13]:
df['lyrics'] = df.lyrics.map(remove_brackets)

### Save/Load

In [14]:
# # uncomment to save
# with gzip.open('data/lyrics_df.pkl', 'wb') as goodbye:
#     pickle.dump(df, goodbye, protocol=pickle.HIGHEST_PROTOCOL)
    
# # uncomment to load
# with gzip.open('data/lyrics_df.pkl', 'rb') as hello:
#     df = pickle.load(hello)

## Rescrapes

### 1960

In [50]:
# rescrape
df.loc[32, 'lyrics'] = lyrics_grabber(access_token, search_term="because they're young duane eddy and the rebels")
df.loc[51, 'lyrics'] = lyrics_grabber(access_token, search_term='jackie wilson night')
df.loc[68, 'lyrics'] = lyrics_grabber(access_token, search_term='mister custer')
df.loc[75, 'lyrics'] = lyrics_grabber(access_token, search_term='the big hurt toni fisher')
df.loc[85, 'lyrics'] = lyrics_grabber(access_token, search_term='Only the Lonely (Know the Way I Feel)')


# more complex ones
soup = soupify('https://www.flashlyrics.com/lyrics/lolita/sailor-your-home-is-the-sea-41')
df.loc[69, 'lyrics'] = '\n'.join(
    [line.contents[0] for line in soup.find('div', attrs={'class': 'main-panel-content'}).find_all('span')
     if line.contents][2:]
)

soup = soupify('https://www.flashlyrics.com/lyrics/the-ivy-three/yogi-79')
df.loc[98, 'lyrics'] = '\n'.join(
    [line.contents[0] for line in soup.find('div', attrs={'class': 'main-panel-content'}).find_all('span')
     if line.contents]
)

In [52]:
# remove lines
df.loc[45, 'lyrics'] = remove_n_lines(df.loc[45, 'lyrics'], 11)

In [53]:
# instrumentals
df.loc[38, 'lyrics'] = np.nan
df.loc[39, 'lyrics'] = np.nan
df.loc[81, 'lyrics'] = np.nan
df.loc[102, 'lyrics'] = np.nan

### 1961

In [80]:
# rescrape
df.loc[106, 'lyrics'] = lyrics_grabber(access_token, 'arthur lyman yellow bird')
df.loc[136, 'lyrics'] = rescrape('https://www.lyricsondemand.com/d/dickanddeedeelyrics/themountainshighlyrics.html',
                                 name='div', attrs={'class': 'lcontent'})
df.loc[174, 'lyrics'] = lyrics_grabber(access_token, "ral donner you don't know what you've got")

# more complex ones
soup = soupify('https://www.flashlyrics.com/lyrics/adam-wade/as-if-i-didnt-know-55')
df.loc[105, 'lyrics'] = '\n'.join(
    [line.contents[0] for line in soup.find('div', attrs={'class': 'main-panel-content'}).find_all('span')
     if line.contents]
)

soup = soupify('https://www.flashlyrics.com/lyrics/ferrante-and-teicher/tonight-12')
df.loc[144, 'lyrics'] = '\n'.join(
    [line.contents[0] for line in soup.find('div', attrs={'class': 'main-panel-content'}).find_all('span')
     if line.contents]
)

In [81]:
# instrumentals
df.loc[110, 'lyrics'] = np.nan
df.loc[145, 'lyrics'] = np.nan
df.loc[161, 'lyrics'] = np.nan
df.loc[185, 'lyrics'] = np.nan
df.loc[198, 'lyrics'] = np.nan

In [82]:
# remove lines
df.loc[105, 'lyrics'] = remove_n_lines(df.loc[105, 'lyrics'], 2)
df.loc[174, 'lyrics'] = remove_n_lines(df.loc[174, 'lyrics'], 5)
df.loc[195, 'lyrics'] = remove_n_lines(df.loc[195, 'lyrics'], 10)
df.loc[208, 'lyrics'] = remove_n_lines(df.loc[208, 'lyrics'], 3)

In [83]:
# update song
df.loc[177, 'title'] = 'Corrine, Corrina'

In [85]:
# remove duplicate
df.drop(index=178, inplace=True)
df.drop(index=182, inplace=True)

### 1962

In [100]:
# rescrape
df.loc[272, 'lyrics'] = lyrics_grabber(access_token, 'joey dee shout')

In [101]:
# instrumentals
df.loc[214, 'lyrics'] = np.nan
df.loc[215, 'lyrics'] = np.nan
df.loc[221, 'lyrics'] = np.nan
df.loc[241, 'lyrics'] = np.nan
df.loc[242, 'lyrics'] = np.nan
df.loc[276, 'lyrics'] = np.nan
df.loc[282, 'lyrics'] = np.nan
df.loc[315, 'lyrics'] = np.nan
df.loc[316, 'lyrics'] = np.nan

In [102]:
# remove lines
df.loc[226, 'lyrics'] = remove_n_lines(df.loc[226, 'lyrics'], 7, 3)
df.loc[256, 'lyrics'] = remove_n_lines(df.loc[256, 'lyrics'], 2)
df.loc[273, 'lyrics'] = remove_n_lines(df.loc[273, 'lyrics'], 5, 1)
df.loc[318, 'lyrics'] = remove_n_lines(df.loc[318, 'lyrics'], 2)

### 1963

In [110]:
# rescrape
df.loc[360, 'lyrics'] = lyrics_grabber(access_token, 'fingertips stevie')

# more complex ones
soup = soupify('https://www.flashlyrics.com/lyrics/dixie-belles/down-at-papa-joes-70')
df.loc[403, 'lyrics'] = '\n'.join(
    [line.contents[0] for line in soup.find('div', attrs={'class': 'main-panel-content'}).find_all('span')
     if line.contents][1:]
)

In [111]:
# instrumentals
df.loc[324, 'lyrics'] = np.nan
df.loc[353, 'lyrics'] = np.nan
df.loc[361, 'lyrics'] = np.nan
df.loc[362, 'lyrics'] = np.nan
df.loc[397, 'lyrics'] = np.nan
df.loc[419, 'lyrics'] = np.nan

In [112]:
# remove lines
df.loc[327, 'lyrics'] = remove_n_lines(df.loc[327, 'lyrics'], 3)
df.loc[344, 'lyrics'] = remove_n_lines(df.loc[344, 'lyrics'], 2)
df.loc[351, 'lyrics'] = remove_n_lines(df.loc[351, 'lyrics'], 2)

### 1964

In [117]:
# rescrape
df.loc[442, 'lyrics'] = lyrics_grabber(access_token, 'summer song chad')
df.loc[473, 'lyrics'] = lyrics_grabber(access_token, 'dancing in the street martha')
df.loc[482, 'lyrics'] = lyrics_grabber(access_token, 'roy orbison pretty woman')
df.loc[518, 'lyrics'] = rescrape('http://www.songlyrics.com/the-larks/jerk-lyrics/',
                                 name='p', attrs={'class': 'songLyricsV14 iComment-text'})

In [118]:
# instrumentals
df.loc[431, 'lyrics'] = np.nan
df.loc[519, 'lyrics'] = np.nan
df.loc[534, 'lyrics'] = np.nan

In [119]:
# remove lines
df.loc[513, 'lyrics'] = remove_n_lines(df.loc[513, 'lyrics'], 6)
df.loc[521, 'lyrics'] = remove_n_lines(df.loc[521, 'lyrics'], 6)

In [120]:
# update artist name
df.loc[442, 'artist'] = 'Chad and Jeremy'

In [121]:
# remove duplicate
df.drop(index=467, inplace=True)

### 1965

In [124]:
# rescrape
df.loc[562, 'lyrics'] = lyrics_grabber(access_token, "Papa’s Got a Brand New Bag (Part 1)")
df.loc[571, 'lyrics'] = lyrics_grabber(access_token, 'nowhere to run martha')
df.loc[593, 'lyrics'] = lyrics_grabber(access_token, 'i got you babe')
df.loc[611, 'lyrics'] = lyrics_grabber(access_token, "I Can’t Help Myself (Sugar Pie, Honey Bunch)")

In [125]:
# instrumentals
df.loc[552, 'lyrics'] = np.nan
df.loc[621, 'lyrics'] = np.nan

### 1966

In [131]:
# rescrape
df.loc[689, 'lyrics'] = lyrics_grabber(access_token, 'good thing paul revere')
df.loc[691, 'lyrics'] = lyrics_grabber(access_token, 'hungry paul revere')
df.loc[692, 'lyrics'] = lyrics_grabber(access_token, 'kicks paul revere')
df.loc[694, 'lyrics'] = lyrics_grabber(access_token, 'lady godiva peter')
df.loc[704, 'lyrics'] = lyrics_grabber(access_token, 'homeward bound simon')
df.loc[735, 'lyrics'] = lyrics_grabber(access_token, 'california dreamin')

In [132]:
# remove duplicate
df.drop(index=690, inplace=True)

### 1967

In [135]:
# rescrape
df.loc[874, 'lyrics'] = rescrape('http://www.songlyrics.com/victor-lundberg/an-open-letter-to-my-teenage-son-lyrics/',
                                 name='p', attrs={'class': 'songLyricsV14 iComment-text'})

### 1968

In [145]:
# rescrape
df.loc[929, 'lyrics'] = lyrics_grabber(access_token, 'mrs. robinson')

In [146]:
# instrumentals
df.loc[888, 'lyrics'] = np.nan
df.loc[904, 'lyrics'] = np.nan
df.loc[905, 'lyrics'] = np.nan
df.loc[919, 'lyrics'] = np.nan

In [140]:
# remove lines
df.loc[899, 'lyrics'] = remove_n_lines(df.loc[899, 'lyrics'], 3, 7)
df.loc[969, 'lyrics'] = remove_n_lines(df.loc[969, 'lyrics'], 5, 1)

### 1969

- There are some top 10 entries that appear as both the single's A and B sides.
- I'll split these up into two separate entries.

In [191]:
%%time

# instantiate empty list
to_add_on = []

# exclude final entry because it is actually a single song
# loop over indices, overwrite row with first song, capture second song
for ind in df[df.title.str.contains(' / ')].index[:-1]:
    second_song = split_combos(df, ind)
    to_add_on.append(second_song)

# convert second songs to dataframe
temp_df = pd.DataFrame(to_add_on)

# combine
df = pd.concat([df, temp_df])

Timeout raised and caught:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)
CPU times: user 8.58 s, sys: 402 ms, total: 8.98 s
Wall time: 4min 9s


In [155]:
# rescrape
df.loc[1003, 'lyrics'] = lyrics_grabber(access_token, 'i want you back the jackson 5')
df.loc[1023, 'lyrics'] = lyrics_grabber(access_token, 'boxer simon garfunkel')
df.loc[1039, 'lyrics'] = lyrics_grabber(access_token, 'beatles get back')

In [156]:
# instrumentals
df.loc[981, 'lyrics'] = np.nan
df.loc[982, 'lyrics'] = np.nan
df.loc[1001, 'lyrics'] = np.nan
df.loc[1057, 'lyrics'] = np.nan
df.loc[1071, 'lyrics'] = np.nan

In [157]:
# remove duplicate
df.drop(index=1038, inplace=True)

### 1970

In [169]:
# rescrape
df.loc[1111, 'lyrics'] = lyrics_grabber(access_token, 'abc the jackson 5')
df.loc[1130, 'lyrics'] = lyrics_grabber(access_token, 'cecilia simon garfunkel')
df.loc[1131, 'lyrics'] = lyrics_grabber(access_token, 'bridge over troubled water simon garfunkel')

### 1971

In [197]:
df.reset_index(drop=True, inplace=True)

In [203]:
# rescrape
df.loc[1172, 'lyrics'] = lyrics_grabber(access_token, 'one toke over the line brewer & shipley')
df.loc[1220, 'lyrics'] = lyrics_grabber(access_token, 'uncle albert')

In [204]:
# edge case, had a slash in song title
df.loc[1225, 'title'] = 'UNCLE ALBERT/ADMIRAL HALSEY'

In [205]:
# instrumentals
df.loc[1183, 'lyrics'] = np.nan

In [206]:
# remove duplicate
df.drop(index=4751, inplace=True)
df.drop(index=4754, inplace=True)

### 1972

In [209]:
# rescrape
df.loc[1337, 'lyrics'] = lyrics_grabber(access_token, 'jungle fever chakachas')

In [210]:
# instrumentals
df.loc[1267, 'lyrics'] = np.nan
df.loc[1275, 'lyrics'] = np.nan
df.loc[1303, 'lyrics'] = np.nan

# edge case; non-lyric song
df.loc[1339, 'lyrics'] = np.nan

In [211]:
# remove duplicate
df.drop(index=1287, inplace=True)

### 1973

In [215]:
# rescrape
df.loc[1412, 'lyrics'] = lyrics_grabber(access_token, 'The Morning After (Theme From The Poseidon Adventure)')

In [216]:
# instrumentals
df.loc[1432, 'lyrics'] = np.nan

# edge case; non-lyric song
df.loc[1392, 'lyrics'] = np.nan

### 1974

In [222]:
# rescrape
df.loc[1497, 'lyrics'] = lyrics_grabber(access_token, 'use my imagination gladys')
df.loc[1515, 'lyrics'] = lyrics_grabber(access_token, 'hollywood swinging')
df.loc[1516, 'lyrics'] = lyrics_grabber(access_token, 'jungle boogie')
df.loc[1531, 'lyrics'] = lyrics_grabber(access_token, 'jet wings')
df.loc[1534, 'lyrics'] = lyrics_grabber(access_token, 'band on the run wings')

In [223]:
# instrumentals
df.loc[1521, 'lyrics'] = np.nan
df.loc[1522, 'lyrics'] = np.nan
df.loc[1553, 'lyrics'] = np.nan
df.loc[4757, 'lyrics'] = np.nan

### 1975

In [229]:
# rescrape
df.loc[1658, 'lyrics'] = lyrics_grabber(access_token, 'yesterday carpenters')
df.loc[1660, 'lyrics'] = lyrics_grabber(access_token, 'philadelphia freedom elton john')
df.loc[1663, 'lyrics'] = lyrics_grabber(access_token, 'fire ohio players')
df.loc[1664, 'lyrics'] = lyrics_grabber(access_token, 'love rollercoaster ohio players')
df.loc[1641, 'lyrics'] = lyrics_grabber(access_token, 'ringo no no song')

In [230]:
# instrumentals
df.loc[1629, 'lyrics'] = np.nan
df.loc[1669, 'lyrics'] = np.nan

### 1976

In [233]:
# rescrape
df.loc[1686, 'lyrics'] = lyrics_grabber(access_token, 'sara smile hall oates')
df.loc[1694, 'lyrics'] = lyrics_grabber(access_token, 'sing a song earth')
df.loc[1730, 'lyrics'] = lyrics_grabber(access_token, 'disco duck rick dees')
df.loc[1738, 'lyrics'] = lyrics_grabber(access_token, 'more more more andrea true')
df.loc[1746, 'lyrics'] = lyrics_grabber(access_token, 'muskrat love')

In [234]:
# instrumentals
df.loc[1729, 'lyrics'] = np.nan
df.loc[1758, 'lyrics'] = np.nan

### 1977

In [242]:
# rescrape
df.loc[1777, 'lyrics'] = lyrics_grabber(access_token, 'bill conti gonna fly now')
df.loc[1843, 'lyrics'] = lyrics_grabber(access_token, 'jet airliner steve miller')
df.loc[1844, 'lyrics'] = lyrics_grabber(access_token, 'eagle steve miller')

In [243]:
# instrumentals
df.loc[1819, 'lyrics'] = np.nan
df.loc[1841, 'lyrics'] = np.nan
df.loc[4769, 'lyrics'] = np.nan

In [244]:
# remove lines
df.loc[1803, 'lyrics'] = remove_n_lines(df.loc[1803, 'lyrics'], 2)

### 1978

In [248]:
# rescrape
df.loc[1867, 'lyrics'] = lyrics_grabber(access_token, 'still the same bob seger')
df.loc[1922, 'lyrics'] = lyrics_grabber(access_token, 'ymca village people')

In [249]:
# instrumentals
df.loc[1873, 'lyrics'] = np.nan

### 1979

In [253]:
# rescrape
df.loc[1952, 'lyrics'] = lyrics_grabber(access_token, 'boogie wonderland')
df.loc[1957, 'lyrics'] = lyrics_grabber(access_token, 'promises eric clapton')
df.loc[1958, 'lyrics'] = lyrics_grabber(access_token, 'watch out for lucy eric clapton')
df.loc[1964, 'lyrics'] = lyrics_grabber(access_token, 'only lonely souther')

In [254]:
# update title
df.loc[1958, 'title'] = 'WATCH OUT FOR LUCY'

In [255]:
# instrumentals
df.loc[1960, 'lyrics'] = np.nan
df.loc[1962, 'lyrics'] = np.nan

In [256]:
# remove duplicate
df.drop(index=4771, inplace=True)
df.drop(index=4772, inplace=True)

In [257]:
# uncomment to save
with gzip.open('data/lyrics_df_updated.pkl', 'wb') as goodbye:
    pickle.dump(df, goodbye, protocol=pickle.HIGHEST_PROTOCOL)
    
# # uncomment to load
# with gzip.open('data/lyrics_df_updated.pkl', 'rb') as hello:
#     df = pickle.load(hello)

In [258]:
df[df.year == 1980]

,year,title,artist,lyrics
2006,1980,ALL OUT OF LOVE,Air Supply,I'm lying alone with my head on the phone\nThi...
2007,1980,LOST IN LOVE,Air Supply,I realize the best part of love is the thinnes...
2008,1980,EVERY WOMAN IN THE WORLD,Air Supply,"Overnight scenes, dinner and wine\nSaturday gi..."
2009,1980,BIGGEST PART OF ME,Ambrosia,Yeah\n\n(Sunrise) There's a new sun a-risin'\n...
2010,1980,DESIRE,Andy Gibb,Ah..... Ah....\nWe may be big or small or blac...
2011,1980,WOMAN IN LOVE,Barbra Streisand,Life is a moment in space\nWhen the dream is g...
2012,1980,GUILTY,Barbra Streisand and Barry Gibb,"Shadows falling, baby, we stand alone\nOut on ..."
2013,1980,THE ROSE,Bette Midler,"Some say love, it is a river\nThat drowns the ..."
2014,1980,YOU MAY BE RIGHT,Billy Joel,"Friday night, I crashed your party\nSaturday, ..."
2015,1980,IT’S STILL ROCK AND ROLL TO ME,Billy Joel,What's the matter with\nThe clothes I'm wearin...


### 1980

In [ ]:
# rescrape
df.loc[1961, 'lyrics'] = lyrics_grabber(access_token, 'another brick in the wall')

In [ ]:
# remove lines
df.loc[1967, 'lyrics'] = remove_n_lines(3, df.loc[1967, 'lyrics'])
df.loc[2013, 'lyrics'] = remove_n_lines(3, df.loc[2013, 'lyrics'])

In [252]:
print(df.loc[1964, 'lyrics'])

I spend time in the hood, Martin luther king Mansion Cross the confusion and the regain orange re-occupied maestro
Reconcerned recognises taurine and still eyes see


O, hierarchy helper nylon lammar, lammar lamp, lace corns to eat, helper nepal a corns to eat
Here master of mandatory , no differentiating. super, super
Oh heiress, heiress under wear lace soft festival list, lace corn's to eat abode maestro pad
Restaurant gastronomy questionnaire
Ogii, rice is grease is, yeast is grill mill flour mailed buiz meal, ill eme heel knock knot eme air canteen
Air canteen, eme long drink litter eme relation xx chupa chups
Eme catch juice kilo gram scandal litter ice, chi-chi
Puff blunt fufu bonc, puffpuff daga adinua amazon, strow malt syrup comfort able antilope grey silicon renew anticipating
Recognising in real consultant reproducing fertilizer paprika seen hanging sydrome egg point paprika
Adopt alum gallon daga, lion, cattle, wire barrow galops gallon tip pen biro, in gross
Genesis adam a

### 1981

In [ ]:
# remove lines
df.loc[2027, 'lyrics'] = remove_n_lines(6, df.loc[2027, 'lyrics'])

In [ ]:
# instrumentals
df.loc[2038, 'lyrics'] = np.nan

### 1982

In [ ]:
# rescrape
df.loc[2117, 'lyrics'] = lyrics_grabber(access_token, 'rock this town stray cats')
df.loc[2130, 'lyrics'] = lyrics_grabber(access_token, 'crimson and clover')

In [ ]:
# instrumentals
df.loc[2153, 'lyrics'] = np.nan
df.loc[2160, 'lyrics'] = np.nan

### 1983

In [ ]:
# rescrape
df.loc[2237, 'lyrics'] = lyrics_grabber(access_token, 'sweet dreams eurythmics')

In [ ]:
# remove lines
df.loc[2179, 'lyrics'] = remove_n_lines(8, df.loc[2179, 'lyrics'])

In [ ]:
# remove duplicate
df.drop(index=2205, inplace=True)

### 1984

In [ ]:
# rescrape
df.loc[2266, 'lyrics'] = lyrics_grabber(access_token, 'state of shock jacksons')
df.loc[2266, 'lyrics'] = lyrics_grabber(access_token, 'state of shock jacksons')

### 1985

In [ ]:
# rescrape
df.loc[2309, 'lyrics'] = lyrics_grabber(access_token, 'the warrior scandal')

In [ ]:
# instrumentals
df.loc[2358, 'lyrics'] = np.nan
df.loc[2405, 'lyrics'] = np.nan

In [ ]:
# remove lines
df.loc[2385, 'lyrics'] = remove_n_lines(1, df.loc[2385, 'lyrics'])

### 1986

In [ ]:
# rescrape
df.loc[2501, 'lyrics'] = lyrics_grabber(access_token, 'your wildest dreams')

### 1987

In [ ]:
# rescrape
df.loc[2557, 'lyrics'] = lyrics_grabber(access_token, 'the one i love rem')
df.loc[2663, 'lyrics'] = lyrics_grabber(access_token, 'got the look prince')

In [ ]:
# instrumentals
df.loc[2646, 'lyrics'] = np.nan

In [ ]:
# update artist
df.loc[2557, 'artist'] = 'R.E.M.'

### 1988

In [ ]:
# rescrape
df.loc[2678, 'lyrics'] = lyrics_grabber(access_token, "she's like the wind patrick swayze")

### 1989

In [ ]:
# rescrape
df.loc[2862, 'lyrics'] = lyrics_grabber(access_token, 'friends jody watley')

### 1990

In [ ]:
# remove lines
df.loc[2917, 'lyrics'] = remove_n_lines(3, df.loc[2917, 'lyrics'])

### 1991 - 1995

In [ ]:
# rescrape
df.loc[3280, 'lyrics'] = lyrics_grabber(access_token, "gangsta's paradise coolio")

### 1996

In [ ]:
# rescrape
df.loc[3371, 'lyrics'] = lyrics_grabber(access_token, 'nobody keith sweat')

In [ ]:
# remove lines
df.loc[3324, 'lyrics'] = remove_n_lines(9, df.loc[3324, 'lyrics'])

In [ ]:
# instrumentals
df.loc[3365, 'lyrics'] = np.nan

In [ ]:
# remove duplicate
df.drop(index=3331, inplace=True)

### 1997

In [ ]:
# rescrape
df.loc[3374, 'lyrics'] = lyrics_grabber(access_token, 'mmmbop')
df.loc[3384, 'lyrics'] = lyrics_grabber(access_token, 'feel so good ma$e')
df.loc[3395, 'lyrics'] = lyrics_grabber(access_token, 'gotham city r. kelly')
df.loc[3411, 'lyrics'] = lyrics_grabber(access_token, 'CAN’T NOBODY HOLD ME DOWN')

### 1998-1999

In [ ]:
# rescrape
df.loc[3493, 'lyrics'] = lyrics_grabber(access_token, 'lost in you garth')
df.loc[3512, 'lyrics'] = lyrics_grabber(access_token, 'smooth santana')

### 2000

In [ ]:
# rescrape
df.loc[3541, 'lyrics'] = lyrics_grabber(access_token, "destiny's child independent")
df.loc[3578, 'lyrics'] = lyrics_grabber(access_token, 'maria santana')

In [ ]:
# remove lines
df.loc[3543, 'lyrics'] = remove_n_lines(3, df.loc[3543, 'lyrics'])

In [ ]:
# remove duplicate
df.drop(index=3579, inplace=True)

### 2001

In [ ]:
# rescrape
df.loc[3597, 'lyrics'] = lyrics_grabber(access_token, "lady marmalade Lil’ Kim, Mýa, Christina Aguilera & P!nk")
df.loc[3616, 'lyrics'] = lyrics_grabber(access_token, 'always on time ja rule')
df.loc[3626, 'lyrics'] = lyrics_grabber(access_token, "i'm real jennifer lopez")

In [ ]:
# update artist
df.loc[3597, 'artist'] = 'Lil’ Kim, Mýa, Christina Aguilera & P!nk'

### 2002

In [ ]:
# rescrape
df.loc[3659, 'lyrics'] = lyrics_grabber(access_token, 'down 4 u irv gotti')

In [ ]:
df[df.year == 2002]

In [ ]:
print(df.loc[3659, 'lyrics'])

### 1998

In [ ]:
df.loc[1324, 'lyrics'] = lyrics_grabber(access_token, 'The Morning After (Theme From The Poseidon Adventure)')

In [ ]:
remove_n_lines(2, df.loc[1386, 'lyrics'])

In [ ]:
df[df.artist == 'The Larks']

In [ ]:
rescrape('http://www.songlyrics.com/the-larks/jerk-lyrics/',
                                 soup.find('p', attrs={'class': 'songLyricsV14 iComment-text'}).contents)

In [ ]:
soup = soupify('http://www.songlyrics.com/the-larks/jerk-lyrics/')

In [ ]:
soup.find('p', attrs={'class': 'songLyricsV14 iComment-text'}).contents

In [ ]:
'\n'.join(
    [line.strip() for line in soup.find('div', attrs={'class': 'inner'}).contents
     if isinstance(line, str) if line.strip()]
)

In [ ]:
soup.find('div', attrs={'class': 'inner'}).contents

In [ ]:
please break this code

In [ ]:
print(df.loc[81, 'lyrics'])

In [ ]:
regex = re.compile(".*?\[(.*?)\]")
result = re.findall(regex, df.loc[1, 'lyrics'])
result

In [ ]:
df['lyrics'] = df.lyrics.map(remove_brackets)
df.lyrics.head(100)

In [ ]:
# graph number of unique songs per year
plt.figure(figsize=(10,6))
plt.plot([len(year) for year in all_top10s])
plt.xticks(ticks=range(0,60,5), labels=range(1960,2021,5))
plt.show()

In [ ]:
# graph number of unique songs per year
plt.figure(figsize=(10,6))
sns.barplot(x=[year for year in range(1960,2020)], y=[len(year) for year in all_top10s])
plt.xticks(ticks=range(0,60,5), labels=range(1960,2021,5))
plt.show()

In [149]:
# two songs combined, so separating here
df.loc[985, 'title'] = 'DOWN ON THE CORNER'
df.loc[985, 'lyrics'] = lyrics_grabber(access_token, 'creedence down on the corner')

df.loc[986, 'title'] = 'FORTUNATE SON'
df.loc[986, 'lyrics'] = lyrics_grabber(access_token, 'creedence fortunate son')

In [168]:
# separating song combos
ccr1 = split_combos(1092)

ccr2 = split_combos(1093)

ccr3 = split_combos(1094)

gh = split_combos(1109)

j5 = split_combos(1113)

sfs = split_combos(1132)

bea = split_combos(1139)

t5s = split_combos(1145)

tgw = split_combos(1149)

addons = [ccr1, ccr2, ccr3, gh, j5, sfs, bea, t5s, tgw]
temp_df = pd.DataFrame(addons)

df = pd.concat([df, temp_df])